In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [5]:
!pip install easyocr
!pip install ultralytics

In [4]:
from ultralytics import YOLO
import cv2
from statistics import mode
from google.colab.patches import cv2_imshow
import easyocr
import pandas as pd
import re

# Load pre-trained YOLO model
model = YOLO('/content/drive/MyDrive/DataScience_with_KS/Licanse_Plate_Detection/license_plate_detector.pt')

# Path to your video file
video_path = "/content/drive/MyDrive/DataScience_with_KS/Licanse_Plate_Detection/003.mp4"

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])
preds = ['']

# Initialize a list to store results
results_list = []

# Read the video
video = cv2.VideoCapture(video_path)

def extract_time_from_frame(frame):
    """
    Extract time from the top-right corner of the frame using OCR.
    """
    time_text = ""
    # Crop the top-right area (time display region)
    time_area = frame[0:50, -300:]  # Adjust this crop area as per your video layout
    result = reader.readtext(time_area)
    for detection in result:
        text = detection[1]
        # Match time format (e.g., 09:20:25 or 09-20-25)
        if re.search(r'\d{2}[:\-]\d{2}[:\-]\d{2}', text):
            time_text = text
            break
    return time_text, time_area

while True:
    ret, frame = video.read()
    if not ret:
        break  # Exit loop when video ends

    # Extract time from the current frame
    current_time, time_area = extract_time_from_frame(frame)

    # Draw a rectangle around the time area
    cv2.rectangle(frame, (frame.shape[1] - 300, 0), (frame.shape[1], 50), (255, 0, 0), 2)
    if current_time:
        # Display the extracted time on the frame
        cv2.putText(frame, current_time, (frame.shape[1] - 290, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Run YOLO model to detect license plates
    results = model(frame)
    for result in results[0].boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        # Draw a rectangle around the detected license plate
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        # Crop the detected license plate area
        cr_img = frame[int(y1):int(y2), int(x1):int(x2)]

        # Perform OCR on the cropped license plate image
        result = reader.readtext(cr_img)
        result_time=reader.readtext(time_area)

        print(result,result_time)
        ocr_res = []
        for k in range(len(result)):
            if (result[k][2]) > 0.3 and len(result[k][1]) > 2:  # Confidence threshold
                ocr_res.append(result[k][1])
        ocr_res = ''.join(ocr_res)

        # Process and display valid OCR results
        if len(ocr_res) > 1:
            preds.append(ocr_res.upper())
            license_plate = mode(preds)  # Get the most frequent result so far

            # Print the detected time and license plate
            print(f"Time: {current_time}, License Plate: {license_plate}")

            # Annotate the license plate on the frame
            cv2.putText(frame, license_plate, (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Append results to the list
            if current_time and license_plate:
                results_list.append({"Time": current_time, "License Plate": license_plate})

    # Display the frame with annotations
    cv2_imshow(frame)

# Release the video object
video.release()


